In [ ]:
!pip install torchvision


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

# Ensure CUDA is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing
    transforms.ToTensor(),          # Convert to Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  # Normalization
    transforms.RandomHorizontalFlip(),  # Data Augmentation: Flipping
    transforms.RandomRotation(15),  # Data Augmentation: Rotation
])

# Load Dataset (for 20 images in a single folder)
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transforms.ToPILImage()(image)

        # Example label assignment
        label = 1 if 'texture' in self.image_files[idx] else 0

        if self.transform:
            image = self.transform(image)
        return image, label

# Define dataset and dataloader
dataset = CustomImageDataset(img_dir="textures", transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load Pretrained Model and Modify
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
num_classes = 2  # Binary classification (textures vs non-textures)
model.fc = nn.Linear(num_features, num_classes)  # Adjust output layer
model = model.to(device)

# Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct, total = 0, 0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader):.4f}, Accuracy: {correct/total:.4f}')

# Train the model
train_model(model, dataloader, criterion, optimizer, epochs=5)
